# Data Splitting and Image Registration

The preprocessing generally consists of the following basic steps:
1. Splitting the video into the before and after puncture phases
2. For each phase:
  1. Initial preprocessing which consists of some or all of the following
    1. Pseudo-flat field correction
    2. Coarse frame alignment
    3. Backgrounds Subtraction
    4. Copping
  2. Frame-by-frame image registration

In [1]:
using DrWatson
@quickactivate()
using Interact, Queryverse
include(srcdir("reg.jl"));

WebIO._IJuliaInit()

In [2]:
# You need to change directory to the directory in which the data resides
cd(datadir("2021-Past","4xgcamp1exp5scap5day8_1_t1"))

In [3]:
"""
Visualize videos using a slider to move between frames.
"""
function visualize(frames)
    @manipulate throttle=0.1 for t=1:size(frames, timedim(frames))
        frames[time=t]
    end
end

visualize

We are going to have a global `force` option to forcibly recompute things.

In [4]:
force = false;

## Loading and Splitting Video

We start out by loading the `video.avi` file, and visualize it to identify the frame at which the puncture happens.

In [5]:
frames = loadframes("video.avi"; color=Gray{N0f16});

In [6]:
visualize(frames)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["t"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 506, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
253, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/HANNAH/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/HANNAH/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/HANNAH/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/HANNAH/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/HANNAH/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000145e728c0, Task (runnable) @0x0000000145e728c0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40

If there is a puncture the video needs to be split into a "before" phase (the frames preceeding the puncture), and the "after" phase which begins some number of frames after the puncture until either the end of the video or some major event (e.g. a subsequent puncture or a bumped table). If you are processing the whole video uninterrupted then continue forward with the frames variable

We visualize these to make sure we are pleased with the split.

In [7]:
# Your job is to choose the ranges below to trim out the puncture
before, after = frames[:,:,1:246], frames[:,:,297:end];

In [ ]:
visualize(before)

In [ ]:
visualize(after)

## Processing the frames before the puncture

In [8]:
beforeregdir = joinpath("before", "registered");

In [9]:
@time beforereg = before |>
    correctorload(beforeregdir; force=true, regularize=true, ffsubtract=true, cropvid=true);

┌ Info: Regularizing Orientation
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:191
Progress: 100%|█████████████████████████████████████████| Time: 0:01:14
┌ Info: Pseudo-Flat Field Correction
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:171
Progress: 100%|█████████████████████████████████████████| Time: 0:02:07
┌ Info: Background subtracting to facilitate cropping; the subtraction will not be present in the final result
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:228
┌ Info: Background Subtraction
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:148
Progress: 100%|█████████████████████████████████████████| Time: 0:03:03
┌ Info: Cropping
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:97
Progress: 100%|█████████████████████████████████████████| Tim

1436.908024 seconds (282.65 M allocations: 231.169 GiB, 10.74% gc time, 2.16% compilation time)


In [10]:
visualize(beforereg)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["t"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 246, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
123, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/HANNAH/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/HANNAH/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/HANNAH/.julia/packages/InteractBase/o1I6G/src/../assets/all.js"), Asset("css", nothing, "/Users/HANNAH/.julia/packages/InteractBase/o1I6G/src/../assets/style.css"), Asset("css", nothing, "/Users/HANNAH/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000013dcd0e80, Task (runnable) @0x000000013dcd0e80), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\",\"36\",\"37\",\"38\",\"39\",\"40

In [ ]:
size(beforereg)

In [11]:
rm(beforeregdir; force=true, recursive=true)
mkpath(beforeregdir)
saveframes(beforeregdir, beforereg);

┌ Info: Writing frames to before/registered
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:34
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


## Process the frames after the puncture

In [12]:
afterregdir = joinpath("after", "registered");

In [13]:
@time afterreg = after |>
    correctorload(afterregdir; force=true, regularize=true, ffsubtract=true, cropvid=true);

┌ Info: Regularizing Orientation
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:191
Progress: 100%|█████████████████████████████████████████| Time: 0:00:53
┌ Info: Pseudo-Flat Field Correction
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:171
Progress: 100%|█████████████████████████████████████████| Time: 0:02:03
┌ Info: Background subtracting to facilitate cropping; the subtraction will not be present in the final result
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:228
┌ Info: Background Subtraction
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:148
Progress: 100%|█████████████████████████████████████████| Time: 0:02:39
┌ Info: Cropping
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:97
Progress: 100%|█████████████████████████████████████████| Tim

872.574008 seconds (191.15 M allocations: 195.623 GiB, 15.90% gc time, 0.02% compilation time)


In [ ]:
visualize(afterreg)

In [ ]:
size(afterreg)

In [14]:
rm(afterregdir; force=true, recursive=true)
mkpath(afterregdir)
saveframes(afterregdir, afterreg);

┌ Info: Writing frames to after/registered
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:34
Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


In [6]:
pafter = frames[:,:,247:end];

In [7]:
pafterregdir = joinpath("pafter", "registered");

In [8]:
@time pafterreg = pafter |>
    correctorload(pafterregdir; force=true, regularize=true, ffsubtract=true, cropvid=true);

┌ Info: Regularizing Orientation
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:191
Progress: 100%|█████████████████████████████████████████| Time: 0:01:16
┌ Info: Pseudo-Flat Field Correction
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:171
Progress: 100%|█████████████████████████████████████████| Time: 0:02:08
┌ Info: Background subtracting to facilitate cropping; the subtraction will not be present in the final result
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:228
┌ Info: Background Subtraction
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:148
Progress: 100%|█████████████████████████████████████████| Time: 0:03:11
┌ Info: Cropping
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:97
Progress: 100%|█████████████████████████████████████████| Tim

1207.954582 seconds (298.85 M allocations: 243.528 GiB, 12.86% gc time, 2.52% compilation time)


In [9]:
rm(pafterregdir; force=true, recursive=true)
mkpath(pafterregdir)
saveframes(pafterregdir, pafterreg);

┌ Info: Writing frames to pafter/registered
└ @ Main /Users/HANNAH/Desktop/Hannah/Elife/Xenobots_VideoPipeline/biobot-punctures/src/reg.jl:34
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
